In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [3]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_youxi_Mugen=pd.DataFrame(columns=columns)
channel_youxi_Mugen

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [4]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [5]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [6]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').click()  #选择频道

In [7]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[6]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'游戏'

In [8]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[9]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[9]/a').text
sub_channel

'Mugen'

In [9]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [10]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_youxi_Mugen.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-16 00:16:48.578101


<ipython-input-10-45677b6a31fa>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-45677b6a31fa>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-45677b6a31fa>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为157411v7d2视频属性的抓取
BV号为157411v7d2的视频属性抓取结束


<ipython-input-10-45677b6a31fa>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为18J411V7gx视频属性的抓取
BV号为18J411V7gx的视频属性抓取结束
开始BV号为18J411b7W1视频属性的抓取
BV号为18J411b7W1的视频属性抓取结束
开始BV号为197411z7uJ视频属性的抓取
BV号为197411z7uJ的视频属性抓取结束
开始BV号为1gJ411p7PR视频属性的抓取
BV号为1gJ411p7PR的视频属性抓取结束
开始BV号为1U7411i7te视频属性的抓取
BV号为1U7411i7te的视频属性抓取结束
开始BV号为14J411b7Rg视频属性的抓取
BV号为14J411b7Rg的视频属性抓取结束
开始BV号为1JJ41177Y2视频属性的抓取
BV号为1JJ41177Y2的视频属性抓取结束
开始BV号为147411i7aH视频属性的抓取
BV号为147411i7aH的视频属性抓取结束
开始BV号为1u7411Y7Gj视频属性的抓取
BV号为1u7411Y7Gj的视频属性抓取结束
开始BV号为1A7411q7H8视频属性的抓取
BV号为1A7411q7H8的视频属性抓取结束
开始BV号为1Z741167Zm视频属性的抓取
BV号为1Z741167Zm的视频属性抓取结束
开始BV号为1g7411e7SZ视频属性的抓取
开始BV号为1g7411e7SZ视频属性的抓取
BV号为1g7411e7SZ的视频属性抓取结束
开始BV号为1PJ41177pJ视频属性的抓取
BV号为1PJ41177pJ的视频属性抓取结束
开始BV号为1VJ411G7ad视频属性的抓取
BV号为1VJ411G7ad的视频属性抓取结束
开始BV号为1KJ411V75P视频属性的抓取
BV号为1KJ411V75P的视频属性抓取结束
开始BV号为1t7411i7we视频属性的抓取
BV号为1t7411i7we的视频属性抓取结束
开始BV号为1AJ411p7AS视频属性的抓取
BV号为1AJ411p7AS的视频属性抓取结束
开始BV号为1VJ411H7Sm视频属性的抓取
BV号为1VJ411H7Sm的视频属性抓取结束
开始BV号为1HJ411p7oa视频属性的抓取
BV号为1HJ411p7oa的视频属性抓取结束
开始BV号为117411Y7ua视频属性的抓取
BV号为117411Y7ua的视频属性抓取结束
开始BV号为1NJ41177F9

...

开始BV号为1zE41157oB视频属性的抓取
BV号为1zE41157oB的视频属性抓取结束
开始BV号为1h7411Q7oZ视频属性的抓取
BV号为1h7411Q7oZ的视频属性抓取结束
开始BV号为1Ft4y1U7Ey视频属性的抓取
BV号为1Ft4y1U7Ey的视频属性抓取结束
开始BV号为13E411s71K视频属性的抓取
开始BV号为13E411s71K视频属性的抓取
BV号为13E411s71K的视频属性抓取结束
开始BV号为1gi4y1n7Hr视频属性的抓取
BV号为1gi4y1n7Hr的视频属性抓取结束
开始BV号为1LE411T7kW视频属性的抓取
BV号为1LE411T7kW的视频属性抓取结束
开始BV号为1RE411L7ic视频属性的抓取
BV号为1RE411L7ic的视频属性抓取结束
开始BV号为1WE411T7tm视频属性的抓取
BV号为1WE411T7tm的视频属性抓取结束
开始BV号为1i7411o7K1视频属性的抓取
BV号为1i7411o7K1的视频属性抓取结束
开始BV号为1664y1M72F视频属性的抓取
BV号为1664y1M72F的视频属性抓取结束
开始BV号为1YT4y157Tc视频属性的抓取
BV号为1YT4y157Tc的视频属性抓取结束
开始BV号为1u7411o7ri视频属性的抓取
BV号为1u7411o7ri的视频属性抓取结束
开始BV号为1b7411X7ix视频属性的抓取
BV号为1b7411X7ix的视频属性抓取结束
开始BV号为1wE411x7q2视频属性的抓取
BV号为1wE411x7q2的视频属性抓取结束
开始BV号为1wE411T7CS视频属性的抓取
BV号为1wE411T7CS的视频属性抓取结束
开始BV号为1gE411u7BC视频属性的抓取
BV号为1gE411u7BC的视频属性抓取结束
开始BV号为1m7411X7Cn视频属性的抓取
BV号为1m7411X7Cn的视频属性抓取结束
开始BV号为1N741197Wg视频属性的抓取
BV号为1N741197Wg的视频属性抓取结束
开始BV号为1AE411E7Ui视频属性的抓取
BV号为1AE411E7Ui的视频属性抓取结束
开始BV号为1og4y1b744视频属性的抓取
BV号为1og4y1b744的视频属性抓取结束
开始BV号为17E411u7pw

BV号为1pK4y1r7Ed的视频属性抓取结束
开始BV号为11V411o7Ka视频属性的抓取
BV号为11V411o7Ka的视频属性抓取结束
开始BV号为1Gi4y187qW视频属性的抓取
BV号为1Gi4y187qW的视频属性抓取结束
开始BV号为1Qg4y187NK视频属性的抓取
BV号为1Qg4y187NK的视频属性抓取结束
开始BV号为1n5411t7qz视频属性的抓取
BV号为1n5411t7qz的视频属性抓取结束
开始BV号为1ai4y187m3视频属性的抓取
BV号为1ai4y187m3的视频属性抓取结束
开始BV号为1PA411879Y视频属性的抓取
BV号为1PA411879Y的视频属性抓取结束
开始BV号为1MA41187af视频属性的抓取
BV号为1MA41187af的视频属性抓取结束
开始BV号为1zQ4y1K779视频属性的抓取
BV号为1zQ4y1K779的视频属性抓取结束
开始BV号为1664y1T7NF视频属性的抓取
BV号为1664y1T7NF的视频属性抓取结束
开始BV号为1Lp4y1y74B视频属性的抓取
BV号为1Lp4y1y74B的视频属性抓取结束
开始BV号为1tZ4y1x7hf视频属性的抓取
BV号为1tZ4y1x7hf的视频属性抓取结束
开始BV号为1y5411p7Ph视频属性的抓取
BV号为1y5411p7Ph的视频属性抓取结束
开始BV号为1GA411q7Dr视频属性的抓取
BV号为1GA411q7Dr的视频属性抓取结束
开始BV号为11a4y1i7zA视频属性的抓取
BV号为11a4y1i7zA的视频属性抓取结束
开始BV号为14g4y1i72j视频属性的抓取
BV号为14g4y1i72j的视频属性抓取结束
开始BV号为1tC4y1p7g8视频属性的抓取
BV号为1tC4y1p7g8的视频属性抓取结束
开始BV号为1QZ4y1s778视频属性的抓取
BV号为1QZ4y1s778的视频属性抓取结束
开始BV号为1Pp4y1X7F8视频属性的抓取
BV号为1Pp4y1X7F8的视频属性抓取结束
开始BV号为1Yg4y1B7Gp视频属性的抓取
BV号为1Yg4y1B7Gp的视频属性抓取结束
开始BV号为1Ui4y14722视频属性的抓取
BV号为1Ui4y14722的视频属性抓取结束
开始BV号为1vp4y1Q7vu

BV号为1Dz4y1Q7VD的视频属性抓取结束
开始BV号为1yz411e7MJ视频属性的抓取
BV号为1yz411e7MJ的视频属性抓取结束
开始BV号为1D5411p7fE视频属性的抓取
BV号为1D5411p7fE的视频属性抓取结束
开始BV号为1Zv411z7Ky视频属性的抓取
BV号为1Zv411z7Ky的视频属性抓取结束
开始BV号为1kT4y177PH视频属性的抓取
BV号为1kT4y177PH的视频属性抓取结束
开始BV号为1pD4y1D7gM视频属性的抓取
BV号为1pD4y1D7gM的视频属性抓取结束
开始BV号为1FV411k7JS视频属性的抓取
BV号为1FV411k7JS的视频属性抓取结束
开始BV号为1jV411k7Mo视频属性的抓取
BV号为1jV411k7Mo的视频属性抓取结束
开始BV号为18f4y117Vd视频属性的抓取
BV号为18f4y117Vd的视频属性抓取结束
开始BV号为1Tk4y1z7fF视频属性的抓取
BV号为1Tk4y1z7fF的视频属性抓取结束
开始BV号为1nz411e7pC视频属性的抓取
BV号为1nz411e7pC的视频属性抓取结束
开始BV号为1cp4y1D7D6视频属性的抓取
BV号为1cp4y1D7D6的视频属性抓取结束
开始BV号为1wD4y1Q7mk视频属性的抓取
BV号为1wD4y1Q7mk的视频属性抓取结束
开始BV号为1Gg4y1v7M2视频属性的抓取
BV号为1Gg4y1v7M2的视频属性抓取结束
开始BV号为1bC4y187oA视频属性的抓取
BV号为1bC4y187oA的视频属性抓取结束
开始BV号为1QK411p7Sb视频属性的抓取
BV号为1QK411p7Sb的视频属性抓取结束
开始BV号为1FC4y1a7uQ视频属性的抓取
BV号为1FC4y1a7uQ的视频属性抓取结束
开始BV号为1QZ4y1H7be视频属性的抓取
开始BV号为1QZ4y1H7be视频属性的抓取
BV号为1QZ4y1H7be的视频属性抓取结束
开始BV号为1Zv411q7BW视频属性的抓取
BV号为1Zv411q7BW的视频属性抓取结束
开始BV号为1KK411W7GL视频属性的抓取
BV号为1KK411W7GL的视频属性抓取结束
开始BV号为1Bp4y1U7Ff视频属性的抓取
BV号为1Bp4y1U7Ff的视

开始BV号为1GT4y1L7HQ视频属性的抓取
BV号为1GT4y1L7HQ的视频属性抓取结束
开始BV号为1s541187eB视频属性的抓取
BV号为1s541187eB的视频属性抓取结束
开始BV号为1TK411M7DH视频属性的抓取
BV号为1TK411M7DH的视频属性抓取结束
开始BV号为1k541187vy视频属性的抓取
BV号为1k541187vy的视频属性抓取结束
开始BV号为1Bt4y1D7Ho视频属性的抓取
BV号为1Bt4y1D7Ho的视频属性抓取结束
开始BV号为17K4y1v79f视频属性的抓取
BV号为17K4y1v79f的视频属性抓取结束
开始BV号为1Sf4y1Q7qo视频属性的抓取
BV号为1Sf4y1Q7qo的视频属性抓取结束
开始BV号为17f4y1Q7dq视频属性的抓取
BV号为17f4y1Q7dq的视频属性抓取结束
开始BV号为1X64y1F7nQ视频属性的抓取
BV号为1X64y1F7nQ的视频属性抓取结束
开始BV号为1FK4y1a7Rx视频属性的抓取
BV号为1FK4y1a7Rx的视频属性抓取结束
开始BV号为1Tt4y1S7z1视频属性的抓取
BV号为1Tt4y1S7z1的视频属性抓取结束
开始BV号为1ZT4y1j7Hm视频属性的抓取
BV号为1ZT4y1j7Hm的视频属性抓取结束
开始BV号为11t4y1D7E5视频属性的抓取
BV号为11t4y1D7E5的视频属性抓取结束
开始BV号为1hD4y1m7hA视频属性的抓取
BV号为1hD4y1m7hA的视频属性抓取结束
开始BV号为1up4y1v7Pb视频属性的抓取
BV号为1up4y1v7Pb的视频属性抓取结束
开始BV号为16p4y1v765视频属性的抓取
BV号为16p4y1v765的视频属性抓取结束
开始BV号为1UA411n7dW视频属性的抓取
BV号为1UA411n7dW的视频属性抓取结束
开始BV号为1EK411n71C视频属性的抓取
BV号为1EK411n71C的视频属性抓取结束
开始BV号为12K411T74U视频属性的抓取
BV号为12K411T74U的视频属性抓取结束
开始BV号为1Pk4y127j2视频属性的抓取
BV号为1Pk4y127j2的视频属性抓取结束
开始BV号为1vf4y1X7vv视频属性的抓取
BV号为1vf4y1X7vv的视

BV号为17a4y1E7kX的视频属性抓取结束
开始BV号为1gi4y1E72H视频属性的抓取
BV号为1gi4y1E72H的视频属性抓取结束
开始BV号为1fh41197rm视频属性的抓取
BV号为1fh41197rm的视频属性抓取结束
开始BV号为1sK411P7zS视频属性的抓取
BV号为1sK411P7zS的视频属性抓取结束
开始BV号为1zA411E79Y视频属性的抓取
BV号为1zA411E79Y的视频属性抓取结束
开始BV号为1CK4y1a7J2视频属性的抓取
BV号为1CK4y1a7J2的视频属性抓取结束
开始BV号为16z4y1f7q3视频属性的抓取
BV号为16z4y1f7q3的视频属性抓取结束
开始BV号为1dv411C7JY视频属性的抓取
BV号为1dv411C7JY的视频属性抓取结束
开始BV号为1K54y1y7bW视频属性的抓取
BV号为1K54y1y7bW的视频属性抓取结束
开始BV号为1L64y1F7PX视频属性的抓取
BV号为1L64y1F7PX的视频属性抓取结束
开始BV号为1eT4y1w78J视频属性的抓取
BV号为1eT4y1w78J的视频属性抓取结束
开始BV号为1Za4y1j7Hb视频属性的抓取
BV号为1Za4y1j7Hb的视频属性抓取结束
开始BV号为1H54y1v79X视频属性的抓取
BV号为1H54y1v79X的视频属性抓取结束
开始BV号为1QZ4y1N7mS视频属性的抓取
BV号为1QZ4y1N7mS的视频属性抓取结束
开始BV号为12A411E7zh视频属性的抓取
BV号为12A411E7zh的视频属性抓取结束
开始BV号为17h41197Lb视频属性的抓取
BV号为17h41197Lb的视频属性抓取结束
开始BV号为13541187AF视频属性的抓取
BV号为13541187AF的视频属性抓取结束
开始BV号为1Yh411X7A2视频属性的抓取
BV号为1Yh411X7A2的视频属性抓取结束
开始BV号为1qk4y1y7CG视频属性的抓取
BV号为1qk4y1y7CG的视频属性抓取结束
开始BV号为1v5411b7Eb视频属性的抓取
BV号为1v5411b7Eb的视频属性抓取结束
开始BV号为1a64y1F7zw视频属性的抓取
BV号为1a64y1F7zw的视频属性抓取结束
开始BV号为1ha4y1E75e

In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [11]:
channel_youxi_Mugen.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [13]:
channel_youxi_Mugen.to_json('channel_youxi_Mugen.json')

In [17]:
pd.set_option('display.max_rows', None)

In [12]:
channel_youxi_Mugen

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
157411v7d2,1月,2020-01-20 08:16:14,L的朋友,无,8万,1366,【MUGEN】十六夜咲夜 VS 迪奥布兰度【1080P】【60帧】,游戏,Mugen,活动作品,...,"[十六夜咲夜, MUGEN, JOJO的奇妙冒险, 游戏, Mugen, mugen, 东方...",11:14,38.0万,1.1万,469,3316,401,1748,833,否
18J411V7gx,1月,2020-01-07 11:35:18,大师牌,无,4.9万,1006,94社长 VS 97大蛇 听说你伤害很97？,游戏,Mugen,普通视频,...,"[MUGEN, 拳皇, 单机游戏, 游戏, Mugen, mugen, 狂级, 社长, KO...",00:53,31.1万,2734,149,410,113,291,195,否
18J411b7W1,1月,2020-01-01 13:50:12,我是太阳之神-,无,6121,34,【mugen】鸣人全形态最新版【附链接下载】,游戏,Mugen,普通视频,...,"[鸣人, MUGEN, 格斗, 火影忍者, 游戏, Mugen, mugen, 手操, 火影...",15:54,18.5万,7758,3838,5816,334,522,1367,否
197411z7uJ,1月,2020-01-29 18:55:12,是你的L恋恋吖,无,395,85,【Mugen】范马勇次郎开满有多强？,游戏,Mugen,普通视频,...,"[刃牙, MUGEN, 游戏, Mugen, mugen, 凶恶, 准神, 范马勇次郎, 杀...",08:54,16.8万,880,32,84,10,80,65,否
1gJ411p7PR,1月,2020-01-11 13:36:52,旺仔史莱姆-,无,5891,256,当超级玛丽遇到破坏神比鲁斯,游戏,Mugen,活动作品,...,"[龙珠, MUGEN, 游戏, Mugen, mugen, 超级马里奥, 毁童年, 比鲁斯,...",02:23,16.1万,1692,120,106,35,94,90,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12K4y187KY,10月,2020-10-02 11:30:30,爱看电视的方风,无,5726,320,6P新版苍白Nao竟被准神五柱之一击破了？,游戏,Mugen,普通视频,...,"[MUGEN, 拳皇, 格斗, 单机游戏, 游戏, Mugen, mugen, KOF, 十...",02:33,9708,319,9,23,3,63,7,否
1bv411k7cm,10月,2020-10-11 07:50:10,咖_啡_猫,无,8,5,八神还能这样连？,游戏,Mugen,普通视频,...,"[八神, 拳皇, 游戏, Mugen, mugen, 单机, 手操, 拳皇wing0.95,...",00:57,9574,159,3,12,0,31,2,否
1ya4y1L7CP,10月,2020-10-26 20:55:41,老牟影视,无,124,66,琦玉:普奇是谁，是下面这个人吗？,游戏,Mugen,普通视频,...,"[一拳超人, MUGEN, 游戏, Mugen, mugen]",01:01,9535,214,3,16,0,20,8,否
1da4y157CM,10月,2020-10-07 16:23:22,xm墨尘,无,18,2,上古观音VS崩坏之抹消者,游戏,Mugen,普通视频,...,"[MUGEN, 游戏, Mugen, mugen, 论外, 凶恶]",00:49,9530,167,14,24,3,23,5,否
